In [1]:
import numpy as np
from numpy.random import seed
seed(42)

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [2]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3
SHADOW_DATASET_SIZE = 1000
ATTACK_TEST_DATASET_SIZE = 4000


target_epochs = 12
attack_epochs = 12
num_shadows = 10

In [3]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")[:10000]
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")[:10000]
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

def target_model_fn():
    """The architecture of the target (victim) model.

    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = tf.keras.models.Sequential()

    model.add(
        layers.Conv2D(
            32,
            (3, 3),
            activation="tanh",
            padding="same",
            input_shape=(WIDTH, HEIGHT, CHANNELS),
        )
    )
    #model.add(layers.Conv2D(32, (3, 3), activation="tanh"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    #model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3), activation="tanh", padding="same"))
    #model.add(layers.Conv2D(64, (3, 3), activation="tanh"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    #model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    model.add(layers.Dense(128, activation="tanh"))
    #model.add(layers.Dropout(0.5))

    model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def attack_model_fn():
    """Attack model that takes target model predictions and predicts membership.

    Following the original paper, this attack model is specific to the class of the input.
    AttachModelBundle creates multiple instances of this model for each class.
    """
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [12]:
(X_train, y_train), (X_test, y_test) = get_data()

# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
target_model.fit(X_train, y_train, epochs=target_epochs, verbose=True)

Training the target model...
Epoch 1/12
10000/10000 [==============================] - 12s 1ms/sample - loss: 1.6906 - acc: 0.3976
Epoch 2/12
10000/10000 [==============================] - 11s 1ms/sample - loss: 1.3236 - acc: 0.5404
Epoch 3/12
10000/10000 [==============================] - 14s 1ms/sample - loss: 1.1436 - acc: 0.5978
Epoch 4/12
10000/10000 [==============================] - 13s 1ms/sample - loss: 0.9770 - acc: 0.6657
Epoch 5/12
10000/10000 [==============================] - 16s 2ms/sample - loss: 0.8551 - acc: 0.7004
Epoch 6/12
10000/10000 [==============================] - 11s 1ms/sample - loss: 0.7194 - acc: 0.7525
Epoch 7/12
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.5633 - acc: 0.8112
Epoch 8/12
10000/10000 [==============================] - 11s 1ms/sample - loss: 0.4366 - acc: 0.8611
Epoch 9/12
10000/10000 [==============================] - 11s 1ms/sample - loss: 0.3003 - acc: 0.9140
Epoch 10/12
10000/10000 [============================

In [5]:
target_model.save("target_cifar10.h5")

In [7]:
from tensorflow.keras.models import load_model

target_model = load_model('target_cifar10.h5')


In [13]:
# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=num_shadows
)

# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.5)
print(attacker_X_train.shape, attacker_X_test.shape)

print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=target_epochs,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test)
    )
)

(5000, 32, 32, 3) (5000, 32, 32, 3)
Training the shadow models...
Train on 1000 samples, validate on 5000 samples
Epoch 1/12
1000/1000 [==============================] - 4s 4ms/sample - loss: 2.0989 - acc: 0.2590 - val_loss: 1.9415 - val_acc: 0.3164
Epoch 2/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.7077 - acc: 0.4200 - val_loss: 1.7723 - val_acc: 0.3732
Epoch 3/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.4389 - acc: 0.5340 - val_loss: 1.7382 - val_acc: 0.3788
Epoch 4/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.2909 - acc: 0.5650 - val_loss: 1.7032 - val_acc: 0.4096
Epoch 5/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.1076 - acc: 0.6390 - val_loss: 1.7638 - val_acc: 0.3940
Epoch 6/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.9691 - acc: 0.6810 - val_loss: 1.7947 - val_acc: 0.4008
Epoch 7/12
1000/1000 [==============================] - 3s 3ms/sample 

Epoch 11/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.2846 - acc: 0.9390 - val_loss: 2.0801 - val_acc: 0.4186
Epoch 12/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.2082 - acc: 0.9600 - val_loss: 2.1954 - val_acc: 0.4102
Train on 1000 samples, validate on 5000 samples
Epoch 1/12
1000/1000 [==============================] - 4s 4ms/sample - loss: 2.0579 - acc: 0.2760 - val_loss: 1.8805 - val_acc: 0.3350
Epoch 2/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.5731 - acc: 0.4600 - val_loss: 1.8106 - val_acc: 0.3588
Epoch 3/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.3722 - acc: 0.5320 - val_loss: 1.7855 - val_acc: 0.3668
Epoch 4/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.2248 - acc: 0.5820 - val_loss: 1.7458 - val_acc: 0.3938
Epoch 5/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.9940 - acc: 0.6760 - val_loss: 1.8404 - val_acc: 0.388

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.5708 - acc: 0.8230 - val_loss: 1.9500 - val_acc: 0.3976
Epoch 10/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.4331 - acc: 0.8910 - val_loss: 2.0101 - val_acc: 0.4030
Epoch 11/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.3280 - acc: 0.9260 - val_loss: 2.1256 - val_acc: 0.3990
Epoch 12/12
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.2354 - acc: 0.9600 - val_loss: 2.1638 - val_acc: 0.4028


In [9]:
from sklearn.metrics import classification_report

# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=attack_epochs, verbose=True)
)

W0623 01:14:13.797880 4659361216 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Training the attack models...
Epoch 1/12
1999/1999 [==============================] - 1s 502us/sample - loss: 0.6137 - acc: 0.6918
Epoch 2/12
1999/1999 [==============================] - 0s 116us/sample - loss: 0.5149 - acc: 0.7359
Epoch 3/12
1999/1999 [==============================] - 0s 71us/sample - loss: 0.5096 - acc: 0.7434
Epoch 4/12
1999/1999 [==============================] - 0s 73us/sample - loss: 0.5095 - acc: 0.7379
Epoch 5/12
1999/1999 [==============================] - 0s 75us/sample - loss: 0.5028 - acc: 0.7444
Epoch 6/12
1999/1999 [==============================] - 0s 92us/sample - loss: 0.5034 - acc: 0.7404
Epoch 7/12
1999/1999 [==============================] - 0s 72us/sample - loss: 0.5046 - acc: 0.7399
Epoch 8/12
1999/1999 [==============================] - 0s 67us/sample - loss: 0.5018 - acc: 0.7404
Epoch 9/12
1999/1999 [==============================] - 0s 66us/sample - loss: 0.5041 - acc: 0.7414
Epoch 10/12
1999/1999 [==============================] - 0s 93us/sam

Epoch 10/12
1974/1974 [==============================] - 0s 115us/sample - loss: 0.5116 - acc: 0.7366
Epoch 11/12
1974/1974 [==============================] - 0s 97us/sample - loss: 0.5131 - acc: 0.7406
Epoch 12/12
1974/1974 [==============================] - 0s 143us/sample - loss: 0.5149 - acc: 0.7366
Epoch 1/12
2033/2033 [==============================] - 1s 437us/sample - loss: 0.5992 - acc: 0.7226
Epoch 2/12
2033/2033 [==============================] - 0s 76us/sample - loss: 0.5087 - acc: 0.7467
Epoch 3/12
2033/2033 [==============================] - 0s 64us/sample - loss: 0.5005 - acc: 0.7521
Epoch 4/12
2033/2033 [==============================] - 0s 64us/sample - loss: 0.5025 - acc: 0.7526
Epoch 5/12
2033/2033 [==============================] - 0s 64us/sample - loss: 0.5001 - acc: 0.7541
Epoch 6/12
2033/2033 [==============================] - 0s 71us/sample - loss: 0.5003 - acc: 0.7501
Epoch 7/12
2033/2033 [==============================] - 0s 66us/sample - loss: 0.4994 - acc: 0

In [17]:
target_model = load_model('non_federated_cifar10.h5')

TypeError: Unexpected keyword argument passed to optimizer: name

In [11]:
# Test the success of the attack.

# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = attacker_X_test[:ATTACK_TEST_DATASET_SIZE], attacker_y_test[:ATTACK_TEST_DATASET_SIZE]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)
print (classification_report(real_membership_labels, attack_guesses))

0.755375
             precision    recall  f1-score   support

        0.0       0.99      0.52      0.68      4000
        1.0       0.67      0.99      0.80      4000

avg / total       0.83      0.76      0.74      8000

